# Customer Review Analysis in NLP Models

## Part 1: Load Data

In [1]:
import numpy as np
import pandas as pd
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\18604\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\18604\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Load data into dataframe
df = pd.read_csv('watch_reviews.tsv', sep='\t', error_bad_lines=False)

C:\Users\18604\AppData\Local\Temp\ipykernel_39104\1838297787.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('watch_reviews.tsv', sep='\t', error_bad_lines=False)
Skipping line 8704: expected 15 fields, saw 22
Skipping line 16933: expected 15 fields, saw 22
Skipping line 23726: expected 15 fields, saw 22

Skipping line 85637: expected 15 fields, saw 22

Skipping line 132136: expected 15 fields, saw 22
Skipping line 158070: expected 15 fields, saw 22
Skipping line 166007: expected 15 fields, saw 22
Skipping line 171877: expected 15 fields, saw 22
Skipping line 177756: expected 15 fields, saw 22
Skipping line 181773: expected 15 fields, saw 22
Skipping line 191085: expected 15 fields, saw 22
Skipping line 196273: expected 15 fields, saw 22
Skipping line 196331: expected 15 fields, saw 22

Skipping line 197000: expected 15 fields, saw 22
Skipping line 197011: expected 15 fie

In [3]:
# Print first 5 rows of the dataset
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [4]:
# Remove missing value
df.dropna(subset = ['review_body'], inplace = True)

In [5]:
df.reset_index(inplace = True, drop = True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960056 entries, 0 to 960055
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960056 non-null  object
 1   customer_id        960056 non-null  int64 
 2   review_id          960056 non-null  object
 3   product_id         960056 non-null  object
 4   product_parent     960056 non-null  int64 
 5   product_title      960054 non-null  object
 6   product_category   960056 non-null  object
 7   star_rating        960056 non-null  int64 
 8   helpful_votes      960056 non-null  int64 
 9   total_votes        960056 non-null  int64 
 10  vine               960056 non-null  object
 11  verified_purchase  960056 non-null  object
 12  review_headline    960049 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960052 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [7]:
# Using the first 1000 data as training data
data = df.loc[:999, 'review_body'].tolist()

## Part 2: Tokenizing and Stemming

In [8]:
# Using NLTK's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("br")
stopwords.append("watch")

print("I use " + str(len(stopwords)) + " stop-words from NLTK library.")
print(stopwords[:10])

I use 183 stop-words from NLTK library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [9]:
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [10]:
data[0]

'Absolutely love this watch! Get compliments almost every time I wear it. Dainty.'

In [11]:
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'wear',
 'dainti']

## Part 3: TF-IDF

In [12]:
tfidf_model = TfidfVectorizer(max_df = 0.99, max_features = 1000,
                              min_df = 0.01, stop_words = 'english',
                              use_idf = True, tokenizer = tokenization_and_stemming, ngram_range = (1,1))

tfidf_matrix = tfidf_model.fit_transform(data)

print("In total, there are " + str(tfidf_matrix.shape[0]) + " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

D:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  warnings.warn(


In total, there are 1000 reviews and 239 terms.


In [13]:
tfidf_matrix.toarray()

array([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [14]:
data[0]

'Absolutely love this watch! Get compliments almost every time I wear it. Dainty.'

In [15]:
tfidf_matrix.toarray()[0]

array([0.        , 0.5125863 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.46967673, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.49124588,
       0.        , 0.        , 0.        , 0.        , 0.     

In [16]:
tfidf_matrix.todense()

matrix([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]])

In [17]:
print(type(tfidf_matrix.toarray()))

<class 'numpy.ndarray'>


In [18]:
print(type(tfidf_matrix.todense()))

<class 'numpy.matrix'>


In [19]:
tf_selected_words = tfidf_model.get_feature_names_out()

In [20]:
tf_selected_words

array(['abl', 'absolut', 'accur', 'actual', 'adjust', 'alarm', 'alreadi',
       'alway', 'amaz', 'amazon', 'anoth', 'arm', 'arriv', 'automat',
       'awesom', 'bad', 'band', 'batteri', 'beauti', 'best', 'better',
       'big', 'bit', 'black', 'blue', 'bought', 'box', 'bracelet',
       'brand', 'break', 'bright', 'broke', 'button', 'buy', 'ca', 'came',
       'case', 'casio', 'chang', 'cheap', 'clasp', 'classi', 'clock',
       'color', 'come', 'comfort', 'compliment', 'cool', 'cost', 'crown',
       'crystal', 'dark', 'date', 'daughter', 'day', 'deal', 'definit',
       'deliveri', 'design', 'dial', 'differ', 'difficult', 'disappoint',
       'display', 'dress', 'durabl', 'easi', 'easili', 'end', 'everi',
       'everyday', 'everyth', 'exact', 'excel', 'expect', 'expens',
       'face', 'fair', 'far', 'fast', 'featur', 'feel', 'fell', 'fine',
       'finish', 'fit', 'function', 'gave', 'gift', 'gold', 'good', 'got',
       'great', 'hand', 'happi', 'hard', 'heavi', 'high', 'hold',
 

## Part 4: K-means clustering

In [21]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters = num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


### 4.1. Analyze K-means Result

In [22]:
# Create DataFrame films from all of the input files.
product = {'review': df[:1000].review_body, 'cluster':clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [23]:
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,0
1,I love this watch it keeps time wonderfully.,0
2,Scratches,1
3,"It works well on me. However, I found cheaper ...",1
4,Beautiful watch face. The band looks nice all...,1
5,"i love this watch for my purpose, about the pe...",0
6,"for my wife and she loved it, looks great and ...",0
7,I was about to buy this thinking it was a Swis...,1
8,Watch is perfect. Rugged with the metal &#34;B...,1
9,Great quality and build.<br />The motors are r...,1


In [24]:
print("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
1,650
0,179
3,77
4,62
2,32


In [25]:
km.cluster_centers_

array([[0.        , 0.02558555, 0.        , ..., 0.00757875, 0.01294037,
        0.00910665],
       [0.0063005 , 0.00448608, 0.00392885, ..., 0.00704477, 0.01683971,
        0.01533387],
       [0.        , 0.        , 0.        , ..., 0.        , 0.02053786,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00851693,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00784247,
        0.        ]])

In [26]:
km.cluster_centers_.shape

(5, 239)

In [28]:
print ("<Document clustering result by K-means>")

order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews) ")

<Document clustering result by K-means>
Cluster 0 words:love,great,look,gift,wife,beauti,
Cluster 0 reviews (179 reviews) 
Cluster 1 words:look,like,work,band,time,perfect,
Cluster 1 reviews (650 reviews) 
Cluster 2 words:excel,product,price,fast,qualiti,good,
Cluster 2 reviews (32 reviews) 
Cluster 3 words:good,qualiti,product,price,look,love,
Cluster 3 reviews (77 reviews) 
Cluster 4 words:nice,price,look,realli,simpl,good,
Cluster 4 reviews (62 reviews) 


## Part 5: Topic Modeling - Latent Dirichlet Allocation (LDA)

In [29]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 5)

In [30]:
# Document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.27341755 0.32146384 0.28453466 0.06008866 0.06049528]
 [0.08449435 0.08467227 0.66197011 0.0846385  0.08422478]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.1000004  0.10000074 0.59950889 0.10006605 0.10042393]
 [0.0667389  0.06772408 0.73151152 0.06714491 0.06688059]
 [0.06756419 0.07038919 0.06845266 0.06919931 0.72439465]]


In [31]:
# Topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 239)
[[ 2.7627634   7.68582745  0.20020369 ...  0.20167435  0.21237825
   2.75668074]
 [ 0.20213522  0.20276187  0.7194353  ...  2.40563039  0.20266574
   0.20048696]
 [ 0.20024139  0.20199805  0.2022761  ...  2.36348708  2.16968994
   2.41429825]
 [ 0.22423501  0.2042434   2.23061754 ...  1.75152441 10.04150699
   7.02190301]
 [ 1.70595205  0.20093206  0.20122246 ...  0.21338108  3.43514688
   0.20373649]]


In [32]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.27,0.32,0.28,0.06,0.06,1
Doc1,0.08,0.08,0.66,0.08,0.08,2
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.50,0.06,0.06,0.32,0.06,0
Doc4,0.08,0.04,0.04,0.51,0.33,3
Doc5,0.08,0.07,0.70,0.07,0.08,2
Doc6,0.07,0.06,0.07,0.06,0.74,4
Doc7,0.06,0.06,0.06,0.06,0.75,4
Doc8,0.30,0.05,0.04,0.57,0.04,3
Doc9,0.06,0.06,0.06,0.06,0.77,4


In [33]:
df_document_topic['topic'].value_counts().to_frame()

,topic
3,284
2,245
4,232
0,163
1,76


In [34]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names_out()
df_topic_words.index = topic_names

df_topic_words.head()

[[ 2.7627634   7.68582745  0.20020369 ...  0.20167435  0.21237825
   2.75668074]
 [ 0.20213522  0.20276187  0.7194353  ...  2.40563039  0.20266574
   0.20048696]
 [ 0.20024139  0.20199805  0.2022761  ...  2.36348708  2.16968994
   2.41429825]
 [ 0.22423501  0.2042434   2.23061754 ...  1.75152441 10.04150699
   7.02190301]
 [ 1.70595205  0.20093206  0.20122246 ...  0.21338108  3.43514688
   0.20373649]]


,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,...,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,2.762763,7.685827,0.200204,0.205020,0.201052,2.378288,0.200817,0.200342,9.505360,0.204562,...,0.200509,0.200865,0.201069,0.204700,0.201056,1.623433,2.561198,0.201674,0.212378,2.756681
Topic1,0.202135,0.202762,0.719435,1.721991,0.201740,0.200104,0.201863,0.204054,0.201150,0.207007,...,4.905223,0.203511,0.695944,0.657893,0.200092,0.212362,0.200355,2.405630,0.202666,0.200487
Topic2,0.200241,0.201998,0.202276,0.201372,0.203610,0.201046,0.204231,0.201921,0.654757,1.522923,...,0.201897,0.200037,0.200607,1.415913,0.200404,23.669104,0.201285,2.363487,2.169690,2.414298
Topic3,0.224235,0.204243,2.230618,2.303291,0.203080,2.959169,4.512821,0.242889,0.205710,2.968600,...,0.207446,1.667585,0.200588,0.347065,3.768172,10.973978,0.209087,1.751524,10.041507,7.021903
Topic4,1.705952,0.200932,0.201222,1.424721,5.483688,0.201804,0.643636,3.990797,0.201858,1.899448,...,0.203288,1.445176,11.344189,1.608138,0.200784,4.582790,1.031168,0.213381,3.435147,0.203736


In [35]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names_out())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,great,beauti,perfect,love,gift,amaz,bought,absolut,pleas,deliveri,fast,function,heavi,purchas,fit
Topic 1,band,light,broke,super,compliment,want,money,happi,thank,wear,piec,weight,time,fell,sturdi
Topic 2,good,nice,love,work,look,qualiti,price,big,time,seller,recommend,pictur,pretti,great,exact
Topic 3,like,band,excel,expect,face,time,color,look,small,read,work,wrist,hand,realli,set
Topic 4,look,great,product,awesom,strap,use,wife,love,comfort,nice,good,day,price,design,water
